In [1]:
import cv2
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk

thres = 0.50  # Threshold to detect objects
is_detection_running = False  # Flag to track if detection is running

# Create the main GUI window
root = tk.Tk()
root.title("Object Detection App")

# Create a label to display the selected image
label = ttk.Label(root)
label.pack(padx=10, pady=10)

# Load class names from coco.names file
classNames = []
classFile = r"coco.names"
with open(classFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

# Load the pre-trained model
configPath = r"ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
weightsPath = r"frozen_inference_graph.pb"

net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

def open_image():
    global is_detection_running
    is_detection_running = True

    # Open a file dialog to select an image
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])

    # Check if a file is selected
    if file_path:
        detect_objects(file_path)

def detect_objects(image_path):
    global is_detection_running

    # Read the image
    img = cv2.imread(image_path)

    # Resize the image to match the expected input size
    img = cv2.resize(img, (700, 500))

    # Detect objects in the image if detection is running
    if is_detection_running:
        classIds, confs, bbox = net.detect(img, confThreshold=thres)

        if len(classIds) != 0:
            for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
                # Ensure classId is within the valid range
                if 0 <= classId < len(classNames):
                    cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
                    cv2.putText(img, classNames[classId - 1].upper(), (box[0] + 10, box[1] + 30),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(img, str(round(confidence * 100, 2)), (box[0] + 200, box[1] + 30),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                else:
                    print(f"Invalid classId: {classId}")

    # Convert the image to RGB format for tkinter
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tk = ImageTk.PhotoImage(Image.fromarray(img_rgb))

    # Update the label with the new image
    label.img = img_tk
    label.config(image=img_tk)

# Open Image button
open_button = ttk.Button(root, text="Open Image", command=open_image)
open_button.pack(side=tk.LEFT, padx=10)

def on_key_press(event):
    # Check if the pressed key is 'q'
    if event.char == 'q':
        root.destroy()

# Bind the key press event to the on_key_press function
root.bind('<KeyPress>', on_key_press)

# Start the GUI main loop
root.mainloop()
